In [1]:
# Importation des bibliothèques nécessaires

import math  # Fournit des fonctions mathématiques standard (ex. : sqrt, pow, etc.).
import random  # Module pour générer des nombres aléatoires (ex. : randint, shuffle, etc.).
import pandas as pd  # Bibliothèque pour manipuler et analyser des données sous forme de tableaux (DataFrames).
from deap import base, creator, tools  
# DEAP est une bibliothèque dédiée aux algorithmes génétiques.
# `base` : fournit des fonctions de base pour construire des algorithmes évolutifs.
# `creator` : permet de définir les classes de fitness et d'individus.
# `tools` : propose des fonctions utilitaires pour les opérations génétiques (sélection, mutation, croisement, etc.).
import numpy as np  # Bibliothèque pour manipuler des tableaux et effectuer des calculs numériques rapides.
from collections import defaultdict  
# Module `collections` :
# `defaultdict` : une sous-classe de dictionnaire qui renvoie une valeur par défaut pour les clés absentes (évite les KeyError).
from IPython.display import display, HTML  
# `display` : permet d'afficher des objets (DataFrames, HTML, etc.) dans un environnement Jupyter Notebook.
# `HTML` : facilite la génération et l'affichage de contenu HTML dans un Notebook.

In [2]:
# Définir une constante RANDOM_STATE pour garantir la reproductibilité des résultats
RANDOM_STATE = 42

# Initialiser le générateur de nombres aléatoires de la bibliothèque random avec la valeur de RANDOM_STATE
# Cela garantit que les nombres aléatoires générés seront les mêmes à chaque exécution du programme
random.seed(RANDOM_STATE)

# Initialiser le générateur de nombres aléatoires de la bibliothèque numpy avec la valeur de RANDOM_STATE
# Cela garantit que les nombres aléatoires générés par numpy seront les mêmes à chaque exécution du programme
np.random.seed(RANDOM_STATE)

In [3]:
# Création de la classe Fitness (fonction d'aptitude) et de l'Individual (chromosome)

# La classe FitnessMin est utilisée pour évaluer la qualité des solutions candidates.
# Le paramètre weights=(-1.0,) indique que l'objectif est de minimiser la valeur de fitness,
# c'est-à-dire minimiser les pénalités.
if not hasattr(creator, "FitnessMin"): # Vérifier si les classes existent déjà avant de les créer
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# La classe Individual représente une solution candidate.
# Chaque individu est une liste de gènes, où chaque gène représente une partie de la solution.
# L'attribut fitness de type FitnessMin permet d'évaluer la qualité de l'individu.
if not hasattr(creator, "Individual"): # Vérifier si les classes existent déjà avant de les créer
    creator.create("Individual", list, fitness=creator.FitnessMin)

# Initialisation de la boîte à outils DEAP

# La Toolbox est un conteneur pour les fonctions et les opérateurs utilisés dans l'algorithme génétique.
# Elle permet de centraliser et de gérer les différentes étapes de l'algorithme, telles que la création
# de la population initiale, l'évaluation des individus, la sélection, le croisement et la mutation.
toolbox = base.Toolbox()

In [4]:
# Générer un individu (chromosome)

# Nombre total de semaines dans l'année (hypothèse : une année complète).
num_weeks = 52

# Nombre de jours travaillés par semaine (lundi à vendredi).
num_days = 5

# Nombre de créneaux horaires disponibles par jour (exemple : matin, milieu de matinée, après-midi, etc.).
num_slots = 4

# Modes possibles pour les cours : "Distanciel" pour en ligne, "Présentiel" pour sur site.
modes = ["Distanciel", "Présentiel"]

# Liste des enseignants disponibles pour dispenser les cours.
teachers = [
    "Farah AIT SALAHT", 
    "Nédra MELLOULI", 
    "Hugo ALATRISTA-SALAS"
]

# Liste des matières enseignées durant l'année.
cours = [
    "AI Algorithms", 
    "Machine Learning", 
    "Programming in data science"
]

# Dictionnaire associant chaque enseignant aux matières qu'il peut enseigner.
teacher_courses = {
    teachers[0]: [cours[0], cours[2]],  # Farah enseigne "AI Algorithms" et "Programming in data science".
    teachers[1]: [cours[1]],           # Nédra enseigne uniquement "Machine Learning".
    teachers[2]: [cours[1], cours[2]]  # Hugo enseigne "Machine Learning" et "Programming in data science".
}

# Liste des groupes d'étudiants (par exemple, différents TD ou promotions).
groupes = [
    "TD A", 
    "TD B", 
    "TD C"
]

# Dictionnaire spécifiant les salles disponibles en fonction du mode de cours.
rooms = {
    "Présentiel": [ # Liste des salles physiques disponibles pour les cours en présentiel.
        "L101", 
        "L102",
        "L103"
    ], 
    "Distanciel": None  # Pas de salle physique requise pour les cours en ligne.
}

# Dictionnaire indiquant combien de fois chaque matière doit être enseignée pour chaque groupe dans l'année.
required_counts = {
    groupes[0]: {cours[0]: 8, cours[1]: 7, cours[2]: 3},
    groupes[1]: {cours[0]: 5, cours[1]: 3, cours[2]: 8},
    groupes[2]: {cours[0]: 2, cours[1]: 6, cours[2]: 4}
}

# Disponibilités des enseignants par semaine, jour, et créneau horaire.
# Exemple : 0 représente le lundi, 1 le mardi, etc. Les créneaux sont indexés de 0 à num_slots-1.
teacher_availability = {
    teachers[0]: {
        1: [1, 2, 3],
        2: [1, 2, 3],
        3: [1, 2, 3, 4],
        4: [1, 2, 3],
        5: [1, 2, 3]
    },
    teachers[1]: {
        1: [2, 3, 4],
        2: [2, 3, 4],
        4: [1, 2, 3, 4],
        5: [1, 2, 3, 4]
    },
    teachers[2]: {
        1: [1, 2, 3, 4],
        2: [1, 2, 3, 4],
        3: [1, 2, 3, 4],
        4: [1, 2, 3, 4],
        5: [1, 2, 3, 4]
    }
}

In [5]:
# Définir un gène (cours) avec des paramètres personnalisables
def create_gene(num_weeks, num_days, num_slots, subjects, teacher_courses, modes, rooms, groups):
    # Générer un numéro de semaine aléatoire entre 1 et num_weeks
    semaine = random.randint(1, num_weeks)

    # Choisir une matière aléatoire parmi les matières disponibles
    matière = random.choice(subjects)
    
    # Trouver les enseignants capables d'enseigner cette matière
    enseignants_possibles = [teacher for teacher, courses in teacher_courses.items() if matière in courses]
    if not enseignants_possibles:
        raise ValueError(f"Aucun enseignant disponible pour enseigner la matière {matière}.")

    # Choisir un enseignant aléatoire parmi ceux qui peuvent enseigner la matière
    enseignant = random.choice(enseignants_possibles)
    
    # Filtrer les jours où l'enseignant est disponible pour cette matière
    jours_disponibles = [jour for jour, créneaux in teacher_availability.get(enseignant, {}).items() if créneaux]
    if not jours_disponibles:
        raise ValueError(f"Aucun jour disponible pour {enseignant} pour enseigner {matière}.")

    # Choisir un jour aléatoire parmi ceux disponibles
    jour = random.choice(jours_disponibles)
    
    # Filtrer les créneaux disponibles pour cet enseignant au jour sélectionné
    créneaux_disponibles = teacher_availability.get(enseignant, {}).get(jour, [])
    if not créneaux_disponibles:
        raise ValueError(f"Aucun créneau disponible pour {enseignant} le jour {jour}.")

    # Choisir un créneau aléatoire parmi ceux disponibles
    créneau = random.choice(créneaux_disponibles)

    # Filtrer les groupes qui ont encore besoin de cours pour cette matière
    groupes_disponibles = [groupe for groupe in groups if required_counts[groupe].get(matière, 0) > 0]
    if not groupes_disponibles:
        raise ValueError(f"Aucun groupe n'a besoin de cours pour la matière {matière}.")

    # Choisir un groupe aléatoire parmi les groupes disponibles
    groupe = random.choice(groupes_disponibles)

    # Choisir un mode de cours aléatoire parmi les modes disponibles
    mode = random.choice(modes)

    # Si le mode est présentiel, choisir une salle aléatoire parmi les salles disponibles, sinon None
    salle = random.choice(rooms[mode]) if rooms[mode] is not None else None

    # Retourner un tuple représentant un gène (cours) avec les informations générées
    return (semaine, jour, créneau, matière, enseignant, mode, groupe, salle)

In [6]:
# Calculer le nombre total de cours requis pour chaque groupe
total_courses_per_group = {group: sum(counts.values()) for group, counts in required_counts.items()}

# Calculer le nombre total de cours requis
total_courses = sum(total_courses_per_group.values())

# Enregistrer la fonction create_gene dans la toolbox avec les paramètres spécifiés pour créer un gène (cours)
# tools.initRepeat est utilisé pour initialiser un individu en répétant la création de gènes
# creator.Individual est la classe représentant un individu
# create_gene est la fonction utilisée pour créer chaque gène
# n=20 indique que chaque individu contient "total_courses" gènes
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 lambda: create_gene(num_weeks, num_days, num_slots, cours, teacher_courses, modes, rooms, groupes), n=total_courses)

# Enregistrer la fonction pour générer une population d'individus
# tools.initRepeat est utilisé pour initialiser une population en répétant la création d'individus
# list est le type de conteneur pour la population
# toolbox.individual est la fonction utilisée pour créer chaque individu
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [7]:
# Définir la fonction d'evaluation
def evaluate(individual, verbose = False):
    penalties = 0  # Initialiser les pénalités à zéro
    
    # Compter le nombre de fois que chaque matière apparaît dans l'individu pour chaque groupe
    actual_counts = {group: {matière: 0 for matière in counts} for group, counts in required_counts.items()}

    # Structure de données pour suivre l'occupation des salles à chaque créneau
    room_occupation = {room: {week: {day: {slot: [] for slot in range(1, num_slots + 1)} for day in range(1, num_days + 1)} for week in range(1, num_weeks + 1)} for room in rooms["Présentiel"]}
    
    # Structure de données pour suivre le nombre de cours par matière et par groupe par jour
    daily_counts = {matière: {group: {(week, day): 0 for week in range(1, num_weeks + 1) for day in range(1, num_days + 1)} for group in groupes} for matière in cours}

    # Structure de données pour suivre le nombre de cours par matière et par groupe par semaine
    weekly_counts = {matière: {group: {week: 0 for week in range(1, num_weeks + 1)} for group in groupes} for matière in cours}

    # Structures de données pour suivre les contraintes TD et professeurs
    td_occupation = {(week, day, slot): [] for week in range(1, num_weeks + 1) for day in range(1, num_days + 1) for slot in range(1, num_slots + 1)}
    teacher_occupation = {(week, day, slot): [] for week in range(1, num_weeks + 1) for day in range(1, num_days + 1) for slot in range(1, num_slots + 1)}

    # Structure pour compter les cours en présentiel et distanciel par matière
    mode_counts = {matière: {'Présentiel': 0, 'Distanciel': 0} for matière in cours}
    
    # Structure pour suivre les cours par groupe, semaine et jour
    day_schedule = {(week, day, group): [] for week in range(1, num_weeks + 1) 
                    for day in range(1, num_days + 1) for group in groupes}
    
    # Structure pour suivre les créneaux occupés par groupe et par jour
    lunch_break_check = {(week, day, groupe): {slot: False for slot in [2, 3, 4]} for week in range(1, num_weeks + 1) for day in range(1, num_days + 1) for groupe in groupes}

    # Boucle sur chaque gène de l'individu pour compter les occurrences de chaque matière pour chaque groupe
    for gene in individual:
        semaine, jour, créneau, matière, enseignant, mode, groupe, salle = gene
        
        # Vérification des contraintes des enseignants avec les matières
        if matière not in teacher_courses.get(enseignant, []):
            if verbose:
                print(f"{enseignant} ne peut pas enseigner {matière} ! (Contrainte + 1)")
            penalties += 1
        
        # Vérification des contraintes TD
        if (semaine, jour, créneau) in td_occupation:
            # Si un autre cours du même groupe existe déjà à ce créneau
            if any(g[6] == groupe for g in td_occupation[(semaine, jour, créneau)]):
                if verbose:
                    print(f"Conflit TD pour le groupe {groupe} ! (Contrainte + 1)")
                penalties += 1
            td_occupation[(semaine, jour, créneau)].append(gene)

        # Vérification des contraintes Professeur
        if (semaine, jour, créneau) in teacher_occupation:
            # Si un autre cours du même enseignant existe déjà à ce créneau
            if any(g[4] == enseignant for g in teacher_occupation[(semaine, jour, créneau)]):
                if verbose:
                    print(f"Conflit Enseignant {enseignant} ! (Contrainte + 1)")
                penalties += 1
            teacher_occupation[(semaine, jour, créneau)].append(gene)

        # Vérifier si la salle est occupée à ce créneau
        if mode == "Présentiel" and salle in room_occupation:
            if (semaine, jour, créneau) in room_occupation[salle]:
                room_occupation[salle][(semaine, jour, créneau)].append(gene)
                if len(room_occupation[salle][(semaine, jour, créneau)]) > 1:
                    if verbose:
                        print("Un salle est en conflit ! (Contrainte + 1)")
                    penalties += 1  # Pénalité pour chaque cours en excès dans la même salle au même créneau

        # Compter les occurrences de chaque matière pour chaque groupe
        if groupe in actual_counts and matière in actual_counts[groupe]:
            actual_counts[groupe][matière] += 1

        # Mise à jour des comptes journaliers et hebdomadaires
        if matière in daily_counts and groupe in daily_counts[matière]:
            daily_counts[matière][groupe][(semaine, jour)] += 1

        if matière in weekly_counts and groupe in weekly_counts[matière]:
            weekly_counts[matière][groupe][semaine] += 1
            
        # Compter les cours en présentiel et distanciel
        mode_counts[matière][mode] += 1
        
        # Marquer les créneaux occupés pour chaque groupe
        if créneau in [2, 3, 4]:
            lunch_break_check[(semaine, jour, groupe)][créneau] = True
            
        # Ajouter le cours dans la structure day_schedule
        day_schedule[(semaine, jour, groupe)].append((créneau, mode))
        
        # Vérification des créneaux disponibles pour l'enseignant
        if enseignant in teacher_availability:
            if jour not in teacher_availability[enseignant] or (créneau not in teacher_availability[enseignant][jour]):
                if verbose:
                    print(f"{enseignant} n'est pas disponible pour le créneau semaine {semaine}, jour {jour}, créneau {créneau}. (Contrainte + 1)")
                penalties += 1

    # Comparer les comptes actuels aux valeurs requises et ajouter des pénalités en conséquence
    for groupe, counts in required_counts.items():
        for matière, required_count in counts.items():
            actual_count = actual_counts[groupe][matière]
            if actual_count < required_count:
                if verbose:
                    print(f"Un cours est manquant ! (Contrainte + {(required_count - actual_count) * 3})")
                penalties += (required_count - actual_count) * 3  # Pénalité pour chaque cours manquant
            elif actual_count > required_count:
                if verbose:
                    print(f"Un cours est en trop ! (Contrainte + {(actual_count - required_count) * 3})")
                penalties += (actual_count - required_count) * 3  # Pénalité pour chaque cours en excès

    # Vérifier la contrainte de maximum un cours par jour par matière et par groupe
    for matière, group_counts in daily_counts.items():
        for group, week_day_counts in group_counts.items():
            for (week, day), count in week_day_counts.items():
                if count > 1:
                    if verbose:
                        print(f"Deux cours de {matière} pour le groupe {group} le même jour (semaine {week}, jour {day}) ! (Contrainte + {count - 1})")
                    penalties += (count - 1)  # Pénalité uniquement pour les doublons dans la même semaine et le même jour

    # Vérifier la contrainte de maximum deux cours par semaine par matière et par groupe
    for matière, group_counts in weekly_counts.items():
        for group, week_counts in group_counts.items():
            for week, count in week_counts.items():
                if count > 2:
                    if verbose:
                        print(f"Trop de cours de la matière {matière} pour la semaine {week} pour le groupe {groupe} ! (Contrainte + {(count - 1)})")
                    penalties += (count - 2)  # Pénalité pour chaque cours en excès par semaine pour ce groupe
                    
    # Vérifier que les cours distanciels ne dépassent pas 50% pour chaque matière
    for matière, counts in mode_counts.items():
        total_cours = counts['Présentiel'] + counts['Distanciel']
        if total_cours > 0:
            distanciel_percentage = (counts['Distanciel'] / total_cours) * 100
            if distanciel_percentage > 50:
                penalty = math.ceil((distanciel_percentage - 50) / 10)  # Pénalité proportionnelle au dépassement
                if verbose:
                    print(f"Trop de cours distanciels pour {matière} ! {distanciel_percentage:.2f}% (Contrainte + {penalty})")
                penalties += penalty
                
    # Vérifier l'existence de la pause déjeuner
    for (week, day, groupe), slots in lunch_break_check.items():
        # Vérifier si tous les créneaux potentiels de pause déjeuner sont occupés
        if all(slots.values()):
            if verbose:
                print(f"Pas de pause déjeuner pour le groupe {groupe} (semaine {week}, jour {day}) ! (Contrainte + 1)")
            penalties += 1
            
    # Vérification de la contrainte présentiel-distanciel consécutif
    for (week, day, group), courses in day_schedule.items():
        # Trier les cours par créneau horaire
        courses.sort(key=lambda x: x[0])  # Trier par créneau (premier élément du tuple)
        for i in range(len(courses)):
            current_slot, current_mode = courses[i]
            # Vérifier le cours précédent
            if i > 0:
                previous_slot, previous_mode = courses[i - 1]
                if current_slot - previous_slot == 1 and current_mode != previous_mode:  # Conflit sur créneaux consécutifs
                    if verbose:
                        print(f"Conflit entre cours {previous_mode} (slot {previous_slot}) et {current_mode} (slot {current_slot}) pour groupe {group}, jour {day}, semaine {week}. (Contrainte + 1)")
                    penalties += 1
            # Vérifier le cours suivant
            if i < len(courses) - 1:
                next_slot, next_mode = courses[i + 1]
                if next_slot - current_slot == 1 and current_mode != next_mode:  # Conflit sur créneaux consécutifs
                    if verbose:
                        print(f"Conflit entre cours {current_mode} (slot {current_slot}) et {next_mode} (slot {next_slot}) pour groupe {group}, jour {day}, semaine {week}. (Contrainte + 1)")
                    penalties += 1

    return (penalties,)

In [8]:
# Enregistrer la fonction d'évaluation dans la boîte à outils
# La fonction evaluate est utilisée pour évaluer la qualité des individus (chromosomes)
# en calculant leur fitness (aptitude).
toolbox.register("evaluate", evaluate)

# Enregistrer la fonction de croisement dans la boîte à outils
# tools.cxTwoPoint est une fonction de croisement à deux points, qui combine deux individus
# en échangeant des segments de leurs gènes pour créer de nouveaux individus.
toolbox.register("mate", tools.cxTwoPoint)  # Croisement à deux points

# Enregistrer la fonction de mutation dans la boîte à outils
# tools.mutShuffleIndexes est une fonction de mutation qui réarrange les gènes d'un individu
# avec une probabilité de 10% (indpb=0.1).
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)  # Mutation avec probabilité de 10%

# Enregistrer la fonction de sélection dans la boîte à outils
# tools.selTournament est une fonction de sélection par tournoi, où un sous-ensemble d'individus
# est choisi aléatoirement et le meilleur individu de ce sous-ensemble est sélectionné.
# tournsize=3 indique que la taille du tournoi est de 3 individus.
toolbox.register("select", tools.selTournament, tournsize=3)  # Sélection par tournoi

In [9]:
# Initialisation de la population
# toolbox.population est utilisé pour générer une population initiale d'individus.
# n=100 indique que la population contient 100 individus.
population = toolbox.population(n=300)

# Algorithme génétique
# NGEN est le nombre de générations pour lesquelles l'algorithme génétique sera exécuté.
NGEN = 100  # Nombre de générations

# CXPB et MUTPB sont les probabilités de croisement et de mutation, respectivement.
# CXPB=0.5 indique que chaque paire d'individus a une probabilité de 50% de subir un croisement.
# MUTPB=0.2 indique que chaque individu a une probabilité de 20% de subir une mutation.
CXPB, MUTPB = 0.5, 0.2  # Probabilité de croisement et de mutation

In [10]:
# Boucle principale de l'algorithme génétique
# L'algorithme s'exécute pour un nombre prédéfini de générations (NGEN)
for gen in range(NGEN):
    # Sélection des individus pour la prochaine génération
    # La fonction de sélection (toolbox.select) est utilisée pour choisir les individus
    # qui participeront à la création de la nouvelle génération.
    # La taille de la nouvelle génération est égale à la taille de la population actuelle.
    offspring = toolbox.select(population, len(population))

    # Clonage des individus sélectionnés pour éviter de modifier la population actuelle
    # La fonction toolbox.clone crée des copies des individus sélectionnés.
    offspring = list(map(toolbox.clone, offspring))

    # Croisement
    # Les individus sont croisés par paires pour créer de nouveaux individus.
    # La probabilité de croisement est définie par CXPB.
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            # Si la condition est remplie, les deux individus sont croisés.
            toolbox.mate(child1, child2)
            # Les valeurs de fitness des nouveaux individus sont effacées
            # car elles doivent être recalculées après le croisement.
            del child1.fitness.values
            del child2.fitness.values

    # Mutation
    # Chaque individu de la nouvelle génération a une probabilité définie par MUTPB
    # de subir une mutation.
    for mutant in offspring:
        if random.random() < MUTPB:
            # Si la condition est remplie, l'individu subit une mutation.
            toolbox.mutate(mutant)
            # Les valeurs de fitness des individus mutés sont effacées
            # car elles doivent être recalculées après la mutation.
            del mutant.fitness.values

    # Évaluation des individus avec une aptitude non calculée
    # Les individus dont la fitness n'est pas valide (c'est-à-dire ceux qui ont été modifiés)
    # sont évalués à nouveau.
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Remplacement de la population par la nouvelle génération
    # La population actuelle est remplacée par la nouvelle génération d'individus.
    population[:] = offspring

In [11]:
# Extraction de la meilleure solution
# Après la fin de toutes les générations, la meilleure solution est extraite de la population.
# La fonction tools.selBest sélectionne le meilleur individu de la population.
best_ind = tools.selBest(population, 1)[0]

In [12]:
# Affiche le résultat de la fonction d'évaluation pour l'individu "best_ind"
print("Nombre de conflits : ", evaluate(best_ind, True))

Nombre de conflits :  (0,)


In [13]:
def afficher_emploi_du_temps(chromosome, groupes=groupes, nb_semaines=num_weeks):
    # Définition des jours de la semaine
    jours_de_la_semaine = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    
    # Définition des créneaux horaires avec leur plage horaire
    heures_des_creneaux = {
        1: "08h00 - 11h00",
        2: "11h00 - 14h00",
        3: "14h00 - 17h00",
        4: "17h00 - 20h00"
    }
    
    # Parcours des semaines
    for semaine in range(nb_semaines):
        # Parcours des groupes définis
        for groupe in groupes:
            # Filtrage des gènes correspondant au groupe et à la semaine actuelle
            groupe_chromosome = [
                g for g in chromosome if g[6] == groupe and g[0] == semaine
            ]
            
            # Initialisation d'un emploi du temps vide avec des dictionnaires imbriqués
            emploi_du_temps = defaultdict(lambda: defaultdict(str))
            
            # Remplissage de l'emploi du temps avec les informations extraites des gènes
            for gene in groupe_chromosome:
                emploi_du_temps[gene[2]][gene[1]] = (
                    f"{gene[3]}<br>"  # Nom du cours
                    f"{gene[4]}<br>"  # Nom de l'enseignant
                    f"{gene[6]}<br>"  # Groupe
                    f"{gene[5]}<br>"  # Type (présentiel ou distanciel)
                    f"{gene[7]}"      # Salle ou lien
                )
            
            # Construction du tableau d'affichage pour le groupe et la semaine actuelle
            tableau = []
            for creneau_index, heure in heures_des_creneaux.items():
                # Ligne correspondant à un créneau horaire
                row = {2: heure}  # Ajout de la plage horaire
                for jour_index in range(len(jours_de_la_semaine)):
                    # Remplissage des cours pour chaque jour
                    row[jours_de_la_semaine[jour_index]] = emploi_du_temps[creneau_index].get(jour_index + 1, "")
                tableau.append(row)  # Ajout de la ligne au tableau
            
            # Affichage de l'emploi du temps pour le groupe et la semaine
            print(f"\n\n=== Groupe {groupe} - Semaine {semaine} ===\n")
            
            # Conversion du tableau en DataFrame pour un affichage structuré
            df = pd.DataFrame(tableau)
            
            # Affichage HTML de la DataFrame avec mise en forme
            display(HTML(df.to_html(escape=False, index=False)))

# Appel de la fonction pour afficher l'emploi du temps de la meilleure solution
afficher_emploi_du_temps(best_ind)



=== Groupe TD A - Semaine 0 ===



2,Lundi,Mardi,Mercredi,Jeudi,Vendredi
08h00 - 11h00,,,,,
11h00 - 14h00,,,,,
14h00 - 17h00,,,,,
17h00 - 20h00,,,,,




=== Groupe TD B - Semaine 0 ===



2,Lundi,Mardi,Mercredi,Jeudi,Vendredi
08h00 - 11h00,,,,,
11h00 - 14h00,,,,,
14h00 - 17h00,,,,,
17h00 - 20h00,,,,,




=== Groupe TD C - Semaine 0 ===



...

2,Lundi,Mardi,Mercredi,Jeudi,Vendredi
08h00 - 11h00,,,,,
11h00 - 14h00,,,,,
14h00 - 17h00,,,,,
17h00 - 20h00,,,,,




=== Groupe TD B - Semaine 51 ===



2,Lundi,Mardi,Mercredi,Jeudi,Vendredi
08h00 - 11h00,,,,,
11h00 - 14h00,,,,,
14h00 - 17h00,,,,,
17h00 - 20h00,,,,,




=== Groupe TD C - Semaine 51 ===



2,Lundi,Mardi,Mercredi,Jeudi,Vendredi
08h00 - 11h00,,,,,
11h00 - 14h00,,,,,
14h00 - 17h00,,,,,
17h00 - 20h00,,,,,


In [14]:
# Créer une liste vide pour stocker les dictionnaires de chaque cours
schedule_data = []

# Parcourir chaque gène dans best_ind
for gene in best_ind:
    # Décomposer le gène en ses composants
    semaine, jour, créneau, matière, enseignant, mode, groupe, salle = gene

    # Convertir le numéro du jour en une chaîne de caractères représentant le jour de la semaine
    jour_str = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"][jour - 1]

    # Si la salle n'est pas spécifiée, utiliser "N/A"
    salle_str = salle if salle else "N/A"

    # Ajouter un dictionnaire avec les informations du cours à la liste schedule_data
    schedule_data.append({
        "Semaine": semaine,
        "Jour": jour_str,
        "Créneau": créneau,
        "Matière": matière,
        "Enseignant": enseignant,
        "Mode": mode,
        "Salle": salle_str,
        "Groupe": groupe
    })

# Créer un DataFrame à partir de la liste de dictionnaires
df = pd.DataFrame(schedule_data)

# Trier les cours par semaine, jour, et créneau pour obtenir un emploi du temps organisé
df = df.sort_values(by=["Semaine", "Jour", "Créneau"])

# Afficher le message "Emploi du temps :"
print("Emploi du temps :")

# Afficher les 50 premières lignes du DataFrame pour vérifier l'emploi du temps
df.head(50)

Emploi du temps :


,Semaine,Jour,Créneau,Matière,Enseignant,Mode,Salle,Groupe
31,1,Jeudi,3,Programming in data science,Hugo ALATRISTA-SALAS,Distanciel,N/A,TD B
43,1,Jeudi,3,AI Algorithms,Farah AIT SALAHT,Présentiel,L102,TD A
20,1,Lundi,2,Machine Learning,Nédra MELLOULI,Distanciel,N/A,TD A
27,2,Mardi,2,Programming in data science,Hugo ALATRISTA-SALAS,Présentiel,L101,TD B
38,5,Lundi,1,AI Algorithms,Farah AIT SALAHT,Distanciel,N/A,TD A
23,6,Mercredi,1,Machine Learning,Hugo ALATRISTA-SALAS,Présentiel,L102,TD A
39,7,Mardi,2,Programming in data science,Farah AIT SALAHT,Présentiel,L101,TD B
34,8,Jeudi,3,AI Algorithms,Farah AIT SALAHT,Présentiel,L103,TD A
21,9,Jeudi,1,Machine Learning,Hugo ALATRISTA-SALAS,Distanciel,N/A,TD C
19,9,Mardi,1,Machine Learning,Hugo ALATRISTA-SALAS,Présentiel,L101,TD C


In [40]:
df = df.reset_index(drop=True)

In [39]:
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl import Workbook

# Définir les couleurs pour distinguer les modes d'enseignement
color_presentiel = PatternFill(start_color="87CEEB", end_color="87CEEB", fill_type="solid")  # Bleu pour présentiel
color_distanciel = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")  # Orange pour distanciel

# Créer un nouveau fichier Excel
wb = Workbook()

# Liste des jours de la semaine et les horaires pour les créneaux
jours_de_la_semaine = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
heures_des_creneaux = {
    1: "08h00 - 11h00",
    2: "11h00 - 14h00",
    3: "14h00 - 17h00",
    4: "17h00 - 20h00"
}

# On génère une feuille par groupe (ex. TD A, TD B)
for groupe in df["Groupe"].unique():
    # Crée une nouvelle feuille avec le nom du groupe
    ws = wb.create_sheet(title=groupe)

    # Ajouter une colonne pour afficher les horaires (toujours dans la première colonne)
    ws.cell(row=2, column=1, value="Horaires").font = Font(bold=True, size=12)
    ws.cell(row=2, column=1).alignment = Alignment(horizontal="center", vertical="center")

    # Ajouter les semaines comme titres au-dessus des jours
    col_offset = 2  # Les colonnes pour les jours commencent après les horaires
    for semaine in sorted(df["Semaine"].unique()):
        # Écrire "Semaine X" en haut, fusionné sur les colonnes des jours
        ws.cell(row=1, column=col_offset, value=f"Semaine {semaine}").font = Font(bold=True, size=12)
        ws.merge_cells(start_row=1, start_column=col_offset, end_row=1, end_column=col_offset + len(jours_de_la_semaine) - 1)

        # Ajouter les noms des jours sous le titre de la semaine
        for col, jour in enumerate(jours_de_la_semaine, start=col_offset):
            ws.cell(row=2, column=col, value=jour).font = Font(bold=True)
            ws.cell(row=2, column=col).alignment = Alignment(horizontal="center", vertical="center")
        # Passer à la section suivante pour les colonnes
        col_offset += len(jours_de_la_semaine) + 1

    # Remplir les créneaux horaires et insérer les cours
    row_offset = 3  # Les créneaux commencent à la ligne 3
    for creneau_id, heure in heures_des_creneaux.items():
        # Écrire l'heure dans la première colonne (colonne des horaires)
        ws.cell(row=row_offset, column=1, value=heure).font = Font(bold=True)
        ws.cell(row=row_offset, column=1).alignment = Alignment(horizontal="center", vertical="center")
        
        # Ajouter les cours pour chaque jour et semaine
        col_offset = 2  # Recommencer à la colonne 2 pour chaque créneau
        for semaine in sorted(df["Semaine"].unique()):
            for jour_index, jour in enumerate(jours_de_la_semaine, start=0):
                # Filtrer les données pour trouver le cours correspondant
                cours = df[
                    (df["Groupe"] == groupe) &
                    (df["Semaine"] == semaine) &
                    (df["Jour"] == jour) &
                    (df["Créneau"] == creneau_id)
                ]
                if not cours.empty:  # Si un cours existe pour ce créneau
                    contenu = "\n".join(
                        f"{row['Matière']} - {row['Enseignant']}"
                        for _, row in cours.iterrows()
                    )
                    # Déterminer le mode (Présentiel ou Distanciel) et la salle
                    mode = cours.iloc[0]["Mode"]
                    salle = cours.iloc[0]["Salle"] if mode == "Présentiel" else "N/A"
                    contenu += f" - {salle}" if mode == "Présentiel" else ""
                    fill_color = color_presentiel if mode == "Présentiel" else color_distanciel
                    # Écrire le cours dans la cellule correspondante
                    cell = ws.cell(row=row_offset, column=col_offset + jour_index, value=contenu)
                    cell.fill = fill_color
                    cell.alignment = Alignment(wrap_text=True, horizontal="center", vertical="center")
            # Passer à la prochaine semaine
            col_offset += len(jours_de_la_semaine) + 1
        # Passer au prochain créneau horaire
        row_offset += 1

# Supprimer la feuille par défaut créée par openpyxl
if "Sheet" in wb.sheetnames:
    del wb["Sheet"]

# Sauvegarder le fichier Excel
file_path = "emploi_du_temps.xlsx"
wb.save(file_path)

file_path


'emploi_du_temps.xlsx'